In [1]:
#| context: setup
from dataclasses import asdict
import pandas as pd
from itables import show
from shiny import render, reactive, ui

from martignac.nomad.entries import (
    get_entries_of_my_uploads,
    get_entries_in_database,
    DEFAULT_DATABASE,
)

def make_clickable(val):
    return '<a target="_blank" href="{}">link</a>'.format(val)

In [2]:
# entries
entries = get_entries_in_database()
df_e = pd.json_normalize({**asdict(e), "url": e.nomad_gui_url} for e in entries)
df_e = df_e[["entry_id", "upload_id", "entry_type", "entry_create_time", "url"]]


# uploads
upload_entries = get_entries_of_my_uploads()
df_u = pd.json_normalize(
    {**asdict(e), "workflow_name": e.workflow_name, "state_point": e.state_point, "mdp_files": e.mdp_files, "url": e.nomad_gui_url} 
    for e in upload_entries if e.comment and e._comment_dict.get("state_point") and e._comment_dict.get("workflow_name")
)
sp_columns = [c for c in df_u.columns if "state_point" in c]
df_u = df_u[
    ["entry_id", "upload_id", "entry_name", "entry_type", "workflow_name", "mdp_files", *sp_columns, "entry_create_time", "url"]
]
df_u["entry_create_time"] = pd.to_datetime(df_u["entry_create_time"]).dt.date

INFO:martignac.nomad.utils:Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/entries/query


INFO:martignac.nomad.entries:retrieving entry IWfLp8VCyT7z9t3BtVy21Q5WZSRW on test server


INFO:martignac.nomad.utils:Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/entries/IWfLp8VCyT7z9t3BtVy21Q5WZSRW


INFO:martignac.nomad.users:retrieving user 30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b on prod server


INFO:martignac.nomad.utils:Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/users/30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b


...

INFO:martignac.nomad.utils:Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/uploads/oWleQmrGQ96m1uu8rnfurg/entries


INFO:martignac.nomad.entries:retrieving entries for upload QKFgx6_bQyGncwz6Pbdiww on test server


INFO:martignac.nomad.utils:Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/uploads/QKFgx6_bQyGncwz6Pbdiww/entries


INFO:martignac.nomad.entries:retrieving entries for upload SIlgD-99Rm-kwsNiEBzB6w on test server


INFO:martignac.nomad.utils:Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/uploads/SIlgD-99Rm-kwsNiEBzB6w/entries


In [3]:
#| content: valuebox
#| title: "Number of entries"
#| icon: plus-circle
#| color: "success"
dict(
  value = len(df_e) + len(df_u)
)

{'value': 14}

In [4]:
#| content: valuebox
#| title: "Dataset ID"
dict(
    value = DEFAULT_DATABASE,
    color = "secondary"
)

{'value': 'HJdEI1q4SV-c5Di43BTT_Q', 'color': 'secondary'}

In [5]:
#| title: "Entries"
@render.table
def df_entries():
    return df_e.style.format({'url': make_clickable})

In [6]:
#| title: "Uploads"
@render.table
def df_uploads():
    return df_u.style.format({'url': make_clickable})

In [7]:
#| title: "Uploads"
show(
    df_u[(df_u["workflow_name"] == "SoluteGenFlow") & (df_u["entry_type"] == "Workflow")]
    .drop(["workflow_name", "state_point.type", "state_point.solvent_name", 
           # "state_point.lambda_state"
          ], axis=1)
    .sort_values("state_point.solute_name")
    .style.format({'url': make_clickable})
)

,entry_id,upload_id,entry_name,entry_type,mdp_files,state_point.solute_name,entry_create_time,url
1,wInlNdcJYGQjDgPVzDKFl-b2WMES,oWleQmrGQ96m1uu8rnfurg,solute_generation.archive.yaml,Workflow,2d7a9e52d14d23e0dfb97192d75a3463,P6,2024-03-21,link


In [8]:
#| title: "Uploads"
show(
    df_u[(df_u["workflow_name"] == "SolventGenFlow") & (df_u["entry_type"] == "Workflow")]
    .drop(["workflow_name", "state_point.type", "state_point.solute_name", 
           # "state_point.lambda_state"
          ], axis=1)
    .sort_values("state_point.solvent_name")
    .style.format({'url': make_clickable})
)

,entry_id,upload_id,entry_name,entry_type,mdp_files,state_point.solvent_name,entry_create_time,url
4,rHIoTH--Q6yh-erHfc7mbSWDczzr,QKFgx6_bQyGncwz6Pbdiww,solvent_generation.archive.yaml,Workflow,ba2c4618058c83ba0ab2d9d7ba76bd1f,HD,2024-03-21,link


In [9]:
#| title: "Uploads"
show(
    df_u[(df_u["workflow_name"] == "SoluteSolvationFlow") & (df_u["entry_type"] == "Workflow") & (df_u["entry_name"] == "full_workflow.solute_solvation.archive.yaml")]
    .drop(["entry_name", "entry_type", "workflow_name", "state_point.type", 
           # "state_point.lambda_state"
          ], axis=1)
    .sort_values("state_point.solvent_name")
    .style.format({'url': make_clickable})
)

,entry_id,upload_id,mdp_files,state_point.solute_name,state_point.solvent_name,entry_create_time,url
7,PIPds3gv6JmVgmUVttXRw6NZuoSY,SIlgD-99Rm-kwsNiEBzB6w,20b07a164f9519df9622051af672c209,P6,HD,2024-03-21,link


In [10]:
#| title: "Uploads"
show(
    df_u[(df_u["workflow_name"] == "AlchemicalTransformationFlow") & (df_u["entry_type"] == "Workflow") & (df_u["entry_name"] == "full_workflow.alchemical_transformation.archive.yaml")]
    .drop(["entry_name", "entry_type", "workflow_name", "state_point.type", 
           # "state_point.lambda_state"
          ], axis=1)
    .sort_values("state_point.solvent_name")
    .style.format({'url': make_clickable})
)

,entry_id,upload_id,mdp_files,state_point.solute_name,state_point.solvent_name,entry_create_time,url
